In [1]:
import requests
from bs4 import BeautifulSoup

from typing import List, Dict, Optional

import pandas as pd
page = requests.get("https://www.15min.lt/maistas/receptai")

soup = BeautifulSoup(page.content, 'html.parser')
all_divs = soup.find_all("div")

recipes_list_div = soup.find("div", class_="wrapper")
all_recipes_div = recipes_list_div.find_all("div", class_="list-row")

results: List[Dict] = []
for recipe_div in all_recipes_div:
    title = recipe_div.find("div", class_="list-row-desc fl").text
    link_to_recipe = recipe_div.find("a")["href"]
    link_to_image = recipe_div.find("img")["src"]

    results.append({
        "title": title.strip(), 
        "link_to_recipe": link_to_recipe,
        "link_to_image": link_to_image
    })

results_df = pd.DataFrame(results)
results_df.to_csv("15_min.csv")

def extract_recipes_info(url: str) -> Optional[List[Dict]]:
    resp = requests.get(url)
    if resp.status_code != 200:
        raise Exception("Cannot reach page!")
    soup = BeautifulSoup(resp.content, 'html.parser')
    recipes_list_div = soup.find("div", class_="wrapper")

    if not recipes_list_div:
        return None    
    all_recipes_div = recipes_list_div.find_all("div", class_="list-row")
    results: List[Dict] = []
    for recipe_div in all_recipes_div:
        title = recipe_div.find("div", class_="list-row-desc fl").text
        link_to_recipe = recipe_div.find("a")["href"]
        link_to_image = recipe_div.find("img")["src"]

        results.append({
            "title": title.strip(), 
            "link_to_recipe": link_to_recipe,
            "link_to_image": link_to_image
        })

    return results

'''def get_recipe_info(url: str) -> Dict:
        resp = requests.get(url)
        soup = BeautifulSoup(resp.content, "html.parser")

        recipe_title = soup.find("div", class_="wrapper").find("h1").text
        main_recipe_image = soup.find("div", class_="image col-18").find("img").get("src")
        #about_recipe = soup.find("div", class_="authorAboutRecipeSegment").text

        ingredients_div = soup.find("div", class_="ingredients")
        #ingredients_table = ingredients_div.find("table")
        #tr_rows = ingredients_table.find_all("tr")
        ingredients: List[Dict] = []
        #try:
        #   for tr_row in tr_rows:
        #      spans = tr_row.find_all("span")
        #     ingredients.append({"ingredient": spans[1].text.strip(), "ammount": spans[0].text.strip()})
        #except IndexError:
        #   pass

        #making_time = soup.find("div", class_="method").find("div", class_="info").text.strip().replace("Paruošimo laikas: ", "")
        making_steps: List[Dict] = []
        description_divs = soup.find("div", class_="description text").find("p", class_="description text").find_all("div", class_="description text")
        for step, description_div in enumerate(description_divs):
            making_steps.append({"step": step + 1, "description": description_div.find("div", class_="text").text})
        
        return {
    "title": recipe_title.strip(),
    "image": main_recipe_image,
    "ingredients": ingredients,
    "making_steps": making_steps
    }
'''

'def get_recipe_info(url: str) -> Dict:\n        resp = requests.get(url)\n        soup = BeautifulSoup(resp.content, "html.parser")\n\n        recipe_title = soup.find("div", class_="wrapper").find("h1").text\n        main_recipe_image = soup.find("div", class_="image col-18").find("img").get("src")\n        #about_recipe = soup.find("div", class_="authorAboutRecipeSegment").text\n\n        ingredients_div = soup.find("div", class_="ingredients")\n        #ingredients_table = ingredients_div.find("table")\n        #tr_rows = ingredients_table.find_all("tr")\n        ingredients: List[Dict] = []\n        #try:\n        #   for tr_row in tr_rows:\n        #      spans = tr_row.find_all("span")\n        #     ingredients.append({"ingredient": spans[1].text.strip(), "ammount": spans[0].text.strip()})\n        #except IndexError:\n        #   pass\n\n        #making_time = soup.find("div", class_="method").find("div", class_="info").text.strip().replace("Paruošimo laikas: ", "")\n        m